In [1]:
import mido

mido.MidiFile(
    "../data/outputs/logs/250307-101520_specdiff_9786345_random/master-recording_250307-101520.mid"
).print_tracks()

=== Track 0
MetaMessage('track_name', name='master-recording_250307-101520.mid', time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('text', text='transition 0 (16.00s)', time=1536)
MetaMessage('text', text='tick 0-1 (16.00s)', time=96)
MetaMessage('text', text='tick 0-2 (17.00s)', time=96)
MetaMessage('text', text='tick 0-3 (18.00s)', time=96)
MetaMessage('text', text='tick 0-4 (19.00s)', time=96)
MetaMessage('text', text='tick 0-5 (20.00s)', time=96)
MetaMessage('text', text='tick 0-6 (21.00s)', time=96)
MetaMessage('text', text='tick 0-7 (22.00s)', time=96)
MetaMessage('text', text='tick 0-8 (23.00s)', time=96)
MetaMessage('text', text='transition 1 (24.00s)', time=2304)
MetaMessage('text', text='tick 1-1 (24.00s)', time=96)
MetaMessage('text', text='tick 1-2 (25.00s)', time=96)
MetaMessage('text', text='tick 1-3 (26.00s)', time=96)
MetaMessage('text'

In [2]:
mido.tempo2bpm(1000000)

60.0

In [1]:
import os
import h5py
import faiss
import numpy as np
from scipy.spatial.distance import cosine

In [2]:
path = "../data/tables/20250110/pitch_histogram.h5"
file1 = "20240227-076-01_0082-0088_t04s00"
file2 = "20240227-076-01_0075-0082_t04s00"
file3 = "20240305-050-03_0000-0009_t09s00"

In [3]:
with h5py.File(path, "r") as hdf:
    print(hdf["filenames"][:5])
    print(hdf["histograms"][:5])
    histograms = np.array(hdf["histograms"], dtype=np.float32)
    filenames = [str(f[0], encoding="utf-8") for f in hdf["filenames"]]
print(filenames[:5])
print(histograms[:5])
histograms /= np.linalg.norm(histograms, axis=1, keepdims=True)
print(histograms[:5])

[[b'20231220-080-01_0000-0005_t00s00']
 [b'20231220-080-01_0000-0005_t00s01']
 [b'20231220-080-01_0000-0005_t00s02']
 [b'20231220-080-01_0000-0005_t00s03']
 [b'20231220-080-01_0000-0005_t00s04']]
[[0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]
 [0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]
 [0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]
 [0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]
 [0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]]
['20231220-080-01_0000-0005_t00s00', '20231220-080-01_0000-0005_t00s01', '20231220-080-01_0000-0005_t00s02', '20231220-080-01_0000-0005_t00s03', '20231220-080

In [4]:
faiss_index = faiss.IndexFlatIP(12)
faiss_index.add(np.array(histograms, dtype=np.float32))  # type: ignore

In [5]:
print(histograms[filenames.index(file1)])
print(histograms[filenames.index(file2)])
print(histograms[filenames.index(file3)])
print(
    1
    - cosine(
        histograms[filenames.index(file1)],
        histograms[filenames.index(file1)],
    ),
    1
    - cosine(
        histograms[filenames.index(file1)],
        histograms[filenames.index(file2)],
    ),
    1
    - cosine(
        histograms[filenames.index(file1)],
        histograms[filenames.index(file3)],
    ),
)

[0.         0.         0.         0.         0.94475734 0.
 0.         0.         0.         0.19357154 0.         0.2645064 ]
[0.         0.         0.         0.         0.9376489  0.
 0.         0.         0.         0.20182417 0.         0.28298685]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
1.0 0.9997699 0.94475734


In [6]:
print(histograms[0])
print(histograms[1])
print(histograms[2])
print(
    1
    - cosine(
        histograms[0],
        histograms[0],
    ),
    1
    - cosine(
        histograms[0],
        histograms[1],
    ),
    1
    - cosine(
        histograms[0],
        histograms[2],
    ),
)

[0.         0.         0.21973239 0.         0.         0.
 0.         0.9755602  0.         0.         0.         0.        ]
[0.         0.         0.21973239 0.         0.         0.
 0.         0.9755602  0.         0.         0.         0.        ]
[0.         0.         0.21973239 0.         0.         0.
 0.         0.9755602  0.         0.         0.         0.        ]
1.0 1.0 1.0


In [9]:
q = np.array(
    [histograms[0]],  # filenames.index(file1)]],
    dtype=np.float32,
)
q /= np.linalg.norm(q, axis=1, keepdims=True)
print(q)
print(filenames.index(file1))
print(faiss_index.is_trained)
print(faiss_index.ntotal)
D, I = faiss_index.search(
    q,
    10,
)  # type: ignore
print(I)
print(D)

[[0.         0.         0.21973239 0.         0.         0.
  0.         0.9755602  0.         0.         0.         0.        ]]
335360
True
545088
[[  7   6   5   4   3   2   1   0 481 480]]
[[1.         1.         1.         1.         1.         1.
  1.         1.         0.99999815 0.99999815]]


In [11]:
histograms[8]

array([0.        , 0.        , 0.        , 0.21973239, 0.        ,
       0.        , 0.        , 0.        , 0.9755602 , 0.        ,
       0.        , 0.        ], dtype=float32)

In [8]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

import numpy as np

d = 64  # dimension
nb = 100000  # database size
nq = 10000  # nb of queries
np.random.seed(1234)  # make reproducible
xb = np.random.random((nb, d)).astype("float32")
xb[:, 0] += np.arange(nb) / 1000.0
xq = np.random.random((nq, d)).astype("float32")
xq[:, 0] += np.arange(nq) / 1000.0

import faiss  # make faiss available

index = faiss.IndexFlatL2(d)  # build the index
print(index.is_trained)
index.add(xb)  # add vectors to the index
print(index.ntotal)

k = 4  # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k)  # sanity check
print(I)
print(D)
D, I = index.search(xq, k)  # actual search
# print(I[:5])  # neighbors of the 5 first queries
# print(I[-5:])  # neighbors of the 5 last queries

True
100000
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.2076297 7.251163 ]
 [0.        6.3235645 6.684581  6.7999454]
 [0.        5.7964087 6.3917365 7.2815123]
 [0.        7.2779055 7.527987  7.6628466]
 [0.        6.7638035 7.2951207 7.368815 ]]
